# Game of life

In [14]:
import tensorflow as tf
import numpy as np

## Game of life rules:
----------------------------------------------
1. If cell is dead and has 3 alive neighbors, it is alive in the next state.
2. If cell is alive and has 2 or 3 neighbors, it stays alive.
3. If cell is alive and has less than 2 neighbors, it will die.
4. If cell is alive and has more than 3 neighbors, it will die.

In [15]:

def next_state(inputs):
    current_state = inputs[0]
    count_alive = inputs[1]
    if current_state == 0 and count_alive == 3:
        return 1
    elif current_state == 1 and (count_alive == 2 or count_alive == 3):
        return 1
    else:
        return 0
    

## Board class
The board class stores one tensor for the board, a kernel (that contains only ones) and a result tensor that stores the temporary convolution.

#### Update:
First we calculate the convolution with the kernel tensor (we add up all the neighbor values). Then we subtract the current value of the cell (so we then have the alive counter for the neighbors). Then we apply the next_state function 
to each pair of self.tensor_res and self.tensor_board and we store the results in self.tensor_board. By this we calculate the next state for each cell. 

--------------------------------------------------

#### Render:
test

---------------------------------------------------------

Reshaping is just there to use the inbuilt tensorflow functions correctly.

In [16]:

class Board:
    def __init__(self, x, y):
        self.tensor_board  = tf.random.uniform(shape=(x, y), dtype=tf.int32, maxval=2)
        self.tensor_kernel = tf.reshape(tf.constant([[1,1,1], [1,1,1], [1,1,1]],
                                             dtype=tf.int32), [3, 3, 1, 1])
        self.tensor_res    = tf.zeros(shape=(x * y,), dtype=tf.int32)

        self.x = x
        self.y = y

    def update(self):
        self.tensor_res   = tf.reshape(tf.nn.convolution(tf.reshape(self.tensor_board, [1, self.x, self.y, 1]),
                                        self.tensor_kernel, padding='SAME'), (self.x * self.y,))
        self.subtract_and_then_apply_next_state()
        
    def subtract_and_then_apply_next_state(self):
        self.tensor_board = tf.reshape(self.tensor_board, shape=(self.x * self.y,))
        self.tensor_res  -= self.tensor_board
        self.tensor_board = tf.map_fn(next_state, elems=tf.transpose([self.tensor_board, self.tensor_res]))
        self.tensor_board = tf.reshape(self.tensor_board, shape=(self.x, self.y))
        
    def render(self):
        print(self.tensor_board)

    def __repr__(self):
        return repr(self.tensor_board)
    
    def __str__ (self):
        return str(self.tensor_board)



In [17]:

board = Board(10, 10)
print(board)
board.update()
print(board)


tf.Tensor(
[[1 1 0 0 0 1 1 0 1 1]
 [1 1 1 1 0 0 1 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]
 [1 0 0 1 0 1 1 0 0 1]
 [1 0 1 0 0 1 1 0 0 0]
 [0 1 1 0 0 1 1 1 0 0]
 [1 1 1 0 0 0 1 1 0 1]
 [0 0 1 1 1 0 1 0 1 1]
 [1 1 1 0 1 0 1 1 1 0]
 [1 1 1 0 1 1 1 0 0 0]], shape=(10, 10), dtype=int32)
tf.Tensor(
[[1 0 0 0 0 1 1 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [1 0 0 1 0 1 1 0 0 0]
 [1 0 1 1 0 1 1 0 0 0]
 [1 0 1 1 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 1]
 [1 0 0 0 0 0 0 0 1 1]
 [1 0 1 0 1 0 1 0 0 0]], shape=(10, 10), dtype=int32)
